In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Precode: Batch_rfg

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [3]:
recfldgrn_list = [
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [4]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX/PNSectSent@Sentence-Tk@TknzLLMGrn
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


# get df_SubUnit

In [5]:
full_recfldgrn_list = ['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
                       'B-P-EC-Diag@Value-DiagDftGrn_idx',
                       'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

full_recfldgrn_list

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [12]:
# to fieldnn.dataflowfn.embedflowfn.py

import pandas as pd

def get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name = 'E'):
    SubUnit_List = []
    for input_name in full_recfldgrn_list:
        d = {}
        d['SubUnitName'] = default_E_subunit_name
        
        recfldgrn = '-'.join(input_name.split('-')[-2:])
        output_layerid = len(input_name.split('-'))
        output_name = input_name.split('Grn')[0]
        
        d['input_names'] = [input_name]
        d['output_name'] = output_name
        
        d['output_layerid'] = output_layerid
        
        SubUnit_List.append(d)
        
    df_SubUnit = pd.DataFrame(SubUnit_List)
    
    return df_SubUnit


In [13]:
default_E_subunit_name = 'E'

df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7


# get df_SubUnit Extra Info

## Get SubUnit Name to NN List

* E --> [E]
* MR --> [M, R]


In [25]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            # print(input_names)
            assert idx != 0
            # assert len(input_names) == 1; not necessary, L can follow M
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

In [26]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
}
#############################################

In [27]:
s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed]
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed]
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed]


## Get Default BasicNN Config List

In [28]:
def get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder):
    
    # (1) get basic information
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    # fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)


    # (3) get vocab information
    # no matter what type of grain, in the end, we will have vocab_tokenizer.
    if 'LLM' in full_recfldgrn:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        init = vocab_tokenizer.name_or_path
    else:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        init = 'random'

    d = {'vocab_tokenizer': vocab_tokenizer, 'init': init}
    return d

In [29]:
def get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                    fldgrn_folder, learner_default_dict):
    
    SubUnit_DefaultBasicNN_List = []
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):
        nn_type, nn_name = nn_type_nn_name.split('-')
        
        if basic_nn_idx == 0:
            # the first BasicNN in the SubUnit
            # expander and merger will only be here.
            # this assigments only works for the first iteration
            # let's make a village rule: only E, R, M can be the first. 
            assert nn_type in ['expander', 'reducer', 'merger']
        
            if nn_type == 'expander':
                input_names_nnlvl = SubUnit_input_names 
                assert len(input_names_nnlvl) == 1
                full_recfldgrn = input_names_nnlvl[0]
                default_para = get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder)
                
            else:
                default_para = {}
            
        else:
            assert nn_type in ['reducer', 'learner']
            if nn_type == 'learner':
                default_para = learner_default_dict[nn_name] # TODO
            else:
                default_para = {}
        
        SubUnit_DefaultBasicNN_List.append(default_para)    
        
    return SubUnit_DefaultBasicNN_List

In [30]:
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed]
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed]
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed]


In [31]:
SubUnit_info = df_SubUnit.iloc[0]
SubUnit_info

SubUnitName                                            E
input_names             [B-P-EC-A1C@V-A1CNumeDftGrn_wgt]
output_name                      B-P-EC-A1C@V-A1CNumeDft
output_layerid                                         5
SubUnit_BasicNN_List                [expander-NumeEmbed]
Name: 0, dtype: object

In [32]:
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 

In [33]:
SubUnit_BasicNN_List

['expander-NumeEmbed']

In [34]:
SubUnit_DefaultBasicNN_List = get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                                              fldgrn_folder, learner_default_dict)

for i in SubUnit_DefaultBasicNN_List:
    print(i.keys())


dict_keys(['vocab_tokenizer', 'init'])


In [35]:

fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s


df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ..."
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing..."
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...


## Get BasicNN Config List

In [36]:
from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para   # ignore this one in this notebook
from fieldnn.configfn.reducerfn import get_reducer_para # ignore this one in this notebook
from fieldnn.configfn.learnerfn import get_learner_para # ignore this one in this notebook



def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        vocab_tokenizer = default_nnpara['vocab_tokenizer']
        init = default_nnpara['init']
        para = get_expander_para(nn_name, default_nnpara, 
                                 embed_size, vocab_tokenizer, init, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1C@DT', 'A1C@V']
        
        fld_childflds = '&'.join([i.split('@')[-1] for i in childflds])
        # fld_childflds: DT&V
        
        output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        # output_prefix: B-P-EC-A1C
        
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config

In [37]:

def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    
    # print('\n\n************** SubUnit_BasicNN_List ****************')
    # print(SubUnit_BasicNN_List)
    # print(SubUnit_input_names)
    # print(SubUnit_output_name)
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]
        
        ##############
        default_nnpara = SubUnit_DefaultBasicNN_List[basic_nn_idx] 
        ##############
        
        
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # print('==========================')
        # print(basic_nn_idx, nn_type_nn_name)
        # print(input_names_nnlvl, '<-------- input_names_nnlvl')
        # print(output_name_nnlvl, '<-------- output_name_nnlvl')
        
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # if not SubUnit_output_name in final_output_name_nnlvl:
    #     print('xxx errors xxx')
    #     print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    #     print(SubUnit_output_name, '<------- SubUnit_output_name')
    #     print('xxx errors xxx')
    assert SubUnit_output_name in final_output_name_nnlvl
    # print('\n************** End SubUnit_BasicNN_List ****************')
    return SubUnit_BasicNN_Config_List


In [38]:
len(SubUnit_BasicNN_List)

1

In [39]:
len(SubUnit_DefaultBasicNN_List)

1

In [40]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   )

SubUnit_BasicNN_Config_List = s

In [41]:
SubUnit_BasicNN_Config_List

[{'nn_type_nn_name': 'expander-NumeEmbed',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-A1C@V-A1CNumeDftGrn_wgt'],
   'output_name_nnlvl': 'B-P-EC-A1C@V-A1CNumeDft',
   'expander_para': {'nn_type': 'expander',
    'nn_name': 'NumeEmbed',
    'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 37},
    'input_size': None,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

In [42]:
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ..."
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing..."
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...


In [43]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

In [44]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit


,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


# SubUnit 

## Module

In [45]:
import torch

from fieldnn.basicnn.expander import Expander_Layer
from fieldnn.basicnn.reducer import Reducer_Layer
from fieldnn.basicnn.merger import Merger_Layer
from fieldnn.basicnn.learner import Learner_Layer

# from ..basicnn.expander import Expander_Layer
# from ..basicnn.reducer import Reducer_Layer
# from ..basicnn.merger import Merger_Layer
# from ..basicnn.learner import Learner_Layer


class SubUnit_Layer(torch.nn.Module):
    '''Currently, it is not latest version'''
    
    def __init__(self, SubUnit_info):
        super(SubUnit_Layer, self).__init__()
        
        # the input names for the SubUnit
        self.SubUnit_input_names = SubUnit_info['input_names']
        
        # the output name for this SubUnit
        self.SubUnit_output_name = SubUnit_info['output_name']
        
        # get the SubUnit's BasicNN Config List
        self.SubUnit_BasicNN_Config_List = SubUnit_info['SubUnit_BasicNN_Config_List']
        
        
        # construct the LayersDict to hold all BasicNN within this SubUnit.
        self.LayersDict = torch.nn.ModuleDict()
        
        # initialize all the BasicNN for this SubUnit.
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            
            # nn_type_nn_name: like reducer-Max, learner-TFM, merger-Merger, expander-llmembed.
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            
            # Basic_Config: the Config for this NN.
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
                
            if 'expander' in nn_type_nn_name:
                expander_para = Basic_Config['expander_para']
                NN = Expander_Layer(input_names_nnlvl, output_name_nnlvl, expander_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'reducer' in nn_type_nn_name:
                reducer_para = Basic_Config['reducer_para']
                NN = Reducer_Layer(input_names_nnlvl, output_name_nnlvl, reducer_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'merger' in nn_type_nn_name:
                merger_para = Basic_Config['merger_para']
                NN = Merger_Layer(input_names_nnlvl, output_name_nnlvl, merger_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
                
            elif 'learner' in nn_type_nn_name:
                learner_para = Basic_Config['learner_para']
                NN = Learner_Layer(input_names_nnlvl, output_name_nnlvl, learner_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            else:
                raise ValueError(f'Current BasicNN {nn_type_nn_name} is not available')


    def forward(self, SubUnit_input_names, RECFLD_TO_TENSOR):

        INPUTS_TO_INFODICT = {}
        
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
            
            NN = self.LayersDict[f'{idx}_{nn_type_nn_name}']
            
            # prepare the input.
            if idx == 0:
                for tensor_name in input_names_nnlvl:
                    INPUTS_TO_INFODICT[tensor_name] = RECFLD_TO_TENSOR[tensor_name]
            else:
                for input_name in input_names_nnlvl:
                    assert input_name in INPUTS_TO_INFODICT
                
            output_name_nnlvl, info_dict = NN(input_names_nnlvl, INPUTS_TO_INFODICT)
            
            # current output will be the input in the next round. 
            INPUTS_TO_INFODICT[output_name_nnlvl] = info_dict
            
        # pick up the SubUnit_output_name and its info_dict
        final_output_name_nnlvl = output_name_nnlvl
        SubUnit_output_name = self.SubUnit_output_name
        if not SubUnit_output_name in final_output_name_nnlvl:
            print(f'Mismatched SubUnit Output and Final NN Output: {final_output_name_nnlvl} vs {output_name_nnlvl}')
        
        info_dict = INPUTS_TO_INFODICT[final_output_name_nnlvl]
        
        return SubUnit_output_name, info_dict
    

## Usage

In [46]:
OutputTensor_2_Embed = {}

for idx, SubUnit_info in df_SubUnit.iterrows():
    # print(SubUnit_info)
    
    output_name = SubUnit_info['output_name']
    SubUnitLayer = SubUnit_Layer(SubUnit_info)
    OutputTensor_2_Embed[output_name] = SubUnitLayer
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
[i for i in batch_rfg]

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [48]:
RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [49]:
OUTPUT_TO_TENSOR = {}

for output_name, SubUnitLayer in OutputTensor_2_Embed.items():
    input_names = SubUnitLayer.SubUnit_input_names
    SubUnit_output_name, info_dict = SubUnitLayer(input_names, RECFLD_TO_TENSOR)
    
    print(SubUnit_output_name)
    OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict

B-P-EC-A1C@V-A1CNumeDft
B-P-EC-Diag@Value-DiagDft
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM


# EmbedBlock

In [50]:
# eventually, we get a df_SubUnit
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


## Module

In [51]:
import torch
# from .subunit import SubUnit_Layer


class EmbedBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(EmbedBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_TENSOR):
        
        OUTPUT_TO_TENSOR = {}
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, RECFLD_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

## Usage

In [52]:
EmbedBlock = EmbedBlockLayer(df_SubUnit)

EmbedBlock

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EmbedBlockLayer(
  (SubUnitDict): ModuleDict(
    (B-P-EC-A1C@V-A1CNumeDft): SubUnit_Layer(
      (LayersDict): ModuleDict(
        (0_expander-NumeEmbed): Expander_Layer(
          (Embed): NumeEmbeddingLayer(
            (embedding): Embedding(37, 128, padding_idx=0)
          )
          (postprocess): ModuleDict(
            (activator): GELU(approximate='none')
            (dropout): Dropout(p=0.5, inplace=False)
            (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          )
        )
      )
    )
    (B-P-EC-Diag@Value-DiagDft): SubUnit_Layer(
      (LayersDict): ModuleDict(
        (0_expander-CateEmbed): Expander_Layer(
          (Embed): CateEmbeddingLayer(
            (embedding): Embedding(801, 128, padding_idx=0)
          )
          (postprocess): ModuleDict(
            (activator): GELU(approximate='none')
            (dropout): Dropout(p=0.5, inplace=False)
            (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
     

In [53]:
RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [54]:
OUTPUT_TO_TENSOR = EmbedBlock(RECFLD_TO_TENSOR)

In [55]:
for k, v in OUTPUT_TO_TENSOR.items():
    print(k, v['info'].shape)

B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])
